# More on Functional Programming

## Using higher-order functions


Collecting results of functions

In [1]:
; A function that returns the square of each element of a list

(defn square [x]
    (* x x))

(defn square-all [numbers]
    (if (empty? numbers)
        nil 
    (cons (square (first numbers))
          (square-all (rest numbers)))))

(square-all [1 2 3 4 5 6])


(1 4 9 16 25 36)

In [2]:
; A function that returns the cube of each element of a list

(defn cube [x]
    (* x x x))

(defn cube-all [numbers]
    (if (empty? numbers)
        ()
    (cons (cube (first numbers))
          (cube-all (rest numbers)))))

(cube-all [1 2 3 4 5 6])

(1 8 27 64 125 216)

In [3]:
; A HOF 'do-to-all' that generalizes tha las 2 functions

(defn do-to-all [func numbers]
    (if (empty? numbers)
        ()
    (cons (func (first numbers))
          (do-to-all func (rest numbers)))))


#'user/do-to-all

In [4]:
; The new function 'do-to-all' can get the square of each element

(do-to-all square [1 2 3 4 5 6])

(1 4 9 16 25 36)

In [5]:
; The new function 'do-to-all' can get the cube of each element

(do-to-all cube [1 2 3 4 5 6])


(1 8 27 64 125 216)

In [6]:
; In summary, 'do-to-all' is a custom implementation of 'map'

(map cube [1 2 3 4 5 6])


(1 8 27 64 125 216)

In [7]:
; But 'do-to-all' blows the stack with large numbers

(do-to-all square (range 11000))


Execution error (StackOverflowError) at user/do-to-all (REPL:6).
null


class java.lang.StackOverflowError: 

In [17]:
; A new implementation of 'do-to-all' using lazy sequences

(defn do-to-all-2 [func numbers]
    (lazy-seq 
        (if (empty? numbers)
            ()
        (cons (func (first numbers))
              (do-to-all-2 func (rest numbers))))))


#'user/do-to-all-2

In [18]:
; Testing with a large number again (doesn't blow the stack anymore)

(take 10 (drop 10000 (do-to-all-2 square (range 11000))))


(100000000 100020001 100040004 100060009 100080016 100100025 100120036 100140049 100160064 100180081)

Reduce list of things

In [19]:
; A function that returns the sum of a list of numbers

(defn total-of [numbers]
    (loop [nums numbers sum 0]
        (if (empty? nums)
            sum
            (recur (rest nums) (+ sum (first nums))))))

(total-of [5 7 9 3 4 1 2 8])

39

In [20]:
; A couple of functions that returns the biggest number of a list of numbers

(defn larger-of [x y]
    (if (> x y) x y))

(defn largest-of [numbers]
    (loop [l numbers candidate (first numbers)]
        (if (empty? l)
            candidate
            (recur (rest l) (larger-of candidate (first l))))))

(largest-of [5 7 9 3 4 1 2 8])

9

In [21]:
; A HOF 'compute-across' that generalizes the last 2 functions

(defn compute-across [func elements value]
    (if (empty? elements)
        value
        (recur func (rest elements) (func value (first elements)))))


#'user/compute-across

In [22]:
; 'compute-across' can compute the sum of a list of numbers

(defn total-of-2 [numbers]
    (compute-across + numbers 0))

(total-of-2 [5 7 9 3 4 1 2 8])

39

In [23]:
; 'compute-across' can also get the biggest number of a list of numbers

(defn largest-of-2 [numbers]
    (compute-across larger-of numbers (first numbers)))

(largest-of [5 7 9 3 4 1 2 8])


9

In [24]:
; ; In summary, 'compute-across' is a custom implementation of 'reduce'

(reduce + [5 7 9 3 4 1 2 8])

39

In [25]:
(reduce max [5 7 9 3 4 1 2 8])


9

Filtering lists of things


In [26]:
; A function that returns a list of the numbers greater than a given number

(defn all-greater-than [threshold numbers]
    (compute-across #(if (> %2 threshold) (conj %1 %2) %1) numbers [])) ; '%' are the anon function args

(all-greater-than 5 [5 7 9 3 4 1 2 8])

[7 9 8]

In [27]:
; A function that returns a list of the numbers smaller than a given number

(defn all-lesser-than [threshold numbers]
    (compute-across #(if (< %2 threshold) (conj %1 %2) %1) numbers []))

(all-lesser-than 5 [5 7 9 3 4 1 2 8])

[3 4 1 2]

In [28]:
; A HOF 'select-if' that generalizes the 2 last functions

(defn select-if [pred elements]
    (compute-across #(if (pred %2) (conj %1 %2) %1) elements []))

#'user/select-if

In [29]:
; 'select-if' can get all the even numbers

(select-if even? [5 7 9 3 4 1 2 8])

[4 2 8]

In [30]:
; 'select-if' can get all the odd numbers

(select-if odd? [5 7 9 3 4 1 2 8])

[5 7 9 3 1]

In [31]:
; 'all-greater-than' can be implemented in terms of 'select'if'

(defn all-greater-than-v2 [threshold numbers]
    (select-if #(> % threshold) numbers))

(all-greater-than-v2 5 [5 7 9 3 4 1 2 8])

[7 9 8]

In [32]:
; 'all-lesser-than' can be implemented in terms of 'select'if'

(defn all-lesser-than-v2 [threshold numbers]
    (select-if #(< % threshold) numbers))


(all-lesser-than-v2 5 [5 7 9 3 4 1 2 8])

[3 4 1 2]

In [33]:
; In summary, 'select-if' is a custom implementation of 'filter'

(filter even? [5 7 9 3 4 1 2 8])

(4 2 8)

In [34]:
(filter odd? [5 7 9 3 4 1 2 8])

(5 7 9 3 1)

## Partial application


In [49]:
; A tax calculator function

(defn price-with-tax [tax-rate amount]
    (->> (/ tax-rate 100)
         (+ 1)
         (* amount)))

(price-with-tax 9.5M 100)


109.500M

In [50]:
; The last function can be used to get the prices
; accounting for California's and New York's tax rates
-
(defn price-with-ca-tax [price]
    (price-with-tax 9.25M price))

(defn price-with-ny-tax [price]
    (price-with-tax 8.0M price))

(def prices [100 200 300 400 500])


#'user/prices

In [51]:
(map price-with-ca-tax prices)


(109.2500M 218.5000M 327.7500M 437.0000M 546.2500M)

In [52]:
(map price-with-ny-tax prices)


(108.00M 216.00M 324.00M 432.00M 540.00M)

In [53]:
; A function 'price-calculator-for-tax' that generalizes 'price-with-ca-tax' and 'price-with-ny-tax'

(defn price-calculator-for-tax [state-tax]
    (fn [price]
        (price-with-tax state-tax price)))




#'user/price-calculator-for-tax

In [54]:
; 'price-with-ca-tax-2' is a 'partially applied' function 
; with CA's tax rate as a partial arg

(def price-with-ca-tax-2 (price-calculator-for-tax 9.25M)) ; Note the function definition as a var

; The results are the same as the 1st version

(map price-with-ca-tax-2 prices)


(109.2500M 218.5000M 327.7500M 437.0000M 546.2500M)

In [55]:
; 'price-with-ny-tax-2' is a 'partially applied' function 
; with NY's tax rate as a partial arg

(def price-with-ny-tax-2 (price-calculator-for-tax 8.0M)) ; Note the function definition as a var
; The results are the same as the 1st version

(map price-with-ny-tax-2 prices)


(108.00M 216.00M 324.00M 432.00M 540.00M)

In [56]:
; A simple function of arity 5 that concatenates the args as a string

(defn of-n-args [a b c d e] 
    (str a b c d e ))


#'user/of-n-args

In [57]:
; Partially applying to the last function the args a, b, c

(defn of-k-args [d e]
    (of-n-args 1 2 3 d e))

(of-k-args \a \b)


"123ab"

In [58]:
; A general form of a function that can partially apply 
; 'n-minus-k-args' to the function 'of-n-args' 

(defn partially-applied [of-n-args & n-minus-k-args]
    (fn [& k-args]
        (apply of-n-args (concat n-minus-k-args k-args))))


#'user/partially-applied

In [59]:
; Partially applying the first 3 args of 'of-n-args'

(def of-2-args (partially-applied of-n-args \a \b \c))

(of-2-args 4 5)


"abc45"

In [60]:
; Partially applying the first 2 args of 'of-n-args'

(def of-3-args (partially-applied of-n-args \a \b))

(of-3-args 3 4 5)


"ab345"

In [61]:
; The function 'of-n-args' is a custom implementation of the 'partial' Clojure function

(def of-2-args (partial of-n-args \a \b \c))

(of-2-args 4 5)

"abc45"

In [62]:
(def of-3-args (partial of-n-args \a \b))

(of-3-args 3 4 5)


"ab345"

In [63]:
; The select-if function defined in the previous section

(defn select-if [pred elements]
    (compute-across #(if (pred %2) (conj %1 %2) %1) elements []))


#'user/select-if

In [64]:
; Rewriting 'select-if' to take an arbitrary initial container

(defn select-into-if [container pred elements]
    (compute-across #(if (pred %2) (conj %1 %2) %1) elements container))

(def numbers [4 9 5 7 6 3 8])


#'user/numbers

In [65]:
; Passing a vector as initial value to 'select-into-if' and evaluating it

(select-into-if [] #(< % 7) numbers)


[4 5 6 3]

In [66]:
; Passing a list as initial value to 'select-into-if' and evaluating it
; (see how the order of the sequence changes)

(select-into-if () #(< % 7) numbers)


(3 6 5 4)

In [67]:
; The 'select-up' function can be defined to return the filtered results in the same order,
; using a vector as the initial container

(def select-up (partial select-into-if []))

(select-up #(< % 9) [5 3 9 6 8])



[5 3 6 8]

In [68]:
; The 'select-down' function can be defined to return the filtered results with the inverted order,
; using a list as the initial container

(def select-down (partial select-into-if ()))

(select-down #(< % 9) [5 3 9 6 8])


(8 6 3 5)

## Closures

Free variables

In [69]:
; Within the context of the inner anonymous function, 
; 'x' is a free variable ; (neither an arg nor a local var)

(defn adder [num1 num2]
    (let [x (+ num1 num2)]
        (fn [y] (+ x y))))


#'user/adder

In [70]:
; A function 'add-5' that 'closes over' the free variable x (5 in this case)

(def add-5 (adder 2 3))

(add-5 10)


15

Delayed computation and closures


In [71]:
; An expression that will execute immediately 
; (will throw divide by zero exception)

(let [x 1 y 0]
    (/ x y))


Execution error (ArithmeticException) at user/eval4329 (REPL:5).
Divide by zero


class java.lang.ArithmeticException: 

In [72]:
; Wrapping the last expression in a try-catch block

(let [x 1 y 0]
    (try
        (/ x y)
    (catch Exception e 
        (println (.getMessage e)))))


Divide by zero


nil

In [73]:
; The last expression can be generalized in a 
; 'try-catch' function 

(defn try-catch [the-try the-catch]
    (try
        (the-try)
    (catch Exception e 
        (the-catch e))))

(let [x 1 y 0]
    (try-catch #(/ x y) 
               #(println (.getMessage %))))


Divide by zero


nil

Closures and objects

In [74]:
; A 'new-user' function that's a closure over the 
; inner anonymous function

(defn new-user [login password email]
    (fn [a]
        (case a
            :login login
            :password-hash (hash password) ; A hidden field
            :email email
            nil)))


#'user/new-user

In [75]:
; The last function bounded to the following args,

(def arjun (new-user "arjun" "secret" "arjun@zololabs.com"))

#'user/arjun

In [76]:
(arjun :login)


"arjun"

In [77]:
(arjun :password-hash) ; Enables information hiding; can use password without letting anyone else see it


1614358358

In [78]:
(arjun :email)


"arjun@zololabs.com"

In [79]:
(arjun :name)


nil

In [80]:
; Rewrite 'new-user' to add behavior in the form of 'authenticate' field

(defn new-user [login password email]
    (fn [a & args]
        (case a
            :login login
            :email email
            :authenticate (= password (first args)))))


#'user/new-user

In [81]:
; The last function bounded to the following args
; (similar to creating an object in OOP languages)

(def adi (new-user "adi" "secret" "adi@currylogic.com"))


#'user/adi

In [82]:
(adi :authenticate "blah")


false

In [83]:
(adi :authenticate "secret")


true

## An object system for Clojure

Defining Classes

In [84]:
; A function that be used as a 'class', in the sense of blueprint for objects

(defn new-class [class-name]
    (fn [command & args]
        (case command
            :name (name class-name))))

; A macro that uses the last function to create the 'defclass' word 
; that simplify the creation of  classes

(defmacro defclass [class-name]
    `(def ~class-name (new-class '~class-name))) ; '~class-name quotes the value of class-name



#'user/defclass

In [85]:
; Creating the class and calling its name field

(defclass Person)

(Person :name)

"Person"

Creating instances

In [86]:
; A function that 'instantiates' a class given as argument

(defn new-object [klass]
    (fn [command & args]
        (case command
            :class klass)))

; Creating a 'cindy' object of 'Person' type

(def cindy (new-object Person))

#'user/cindy

In [87]:
; The new object doesn't have an informative name

(new-object Person)


#function[user/new-object/fn--4372]

In [88]:
; But its class does have an informative name

((cindy :class) :name)


"Person"

In [89]:
; Adding a 'class-name' attribute to the 'new-object' function

(defn new-object [klass]
    (fn [command & args]
        (case command
            :class klass
            :class-name (klass :name))))

(def cindy (new-object Person))
(cindy :class-name)

"Person"

In [90]:
; A function that emulates the 'new' keyword used in some OOP languages

(defn new-class [class-name]
    (fn klass [command & args]
        (case command
            :name (name class-name)
            :new (new-object klass))))


#'user/new-class

In [91]:
(defclass Person)

(def nancy (Person :new))

(nancy :class-name)

"Person"

Objects and state

In [92]:
; An 'object' with get and set! methods (getters and setters in OOP land)

(defn new-object [klass]
    (let [state (ref {})]
        (fn [command & args]
            (case command
                :class klass
                :class-name (klass :name)
                :set! (let [[k v] args] ; The ! means a state change is involved
                    (dosync (alter state assoc k v))
                    nil)
                :get (let [[key] args]
                    (@state key))))))


#'user/new-object

In [93]:
; After instantiating the 'nancy' object, its name field 
; should be empty since it wasn't specified at creation

(def nancy (Person :new))
(nancy :get :name)


nil

In [94]:
; The set! method in action

(nancy :set! :name "Nancy Drew")

nil

In [95]:
; The get method now return a valid name

(nancy :get :name)

"Nancy Drew"

Defining methods

In [96]:
; The function 'method-spec' creates a vector containing the name of the method definition (as a keyword)
; and another s-expression, which can later be evaluated to create an anonymous function

(defn method-spec [sexpr]
    (let [name (keyword (second sexpr))
          body (next sexpr)]
        [name (conj body 'fn)]))

; Example

(method-spec '(method age [] (* 2 10)))


[:age (fn age [] (* 2 10))]

In [97]:
; The 'method-specs' function pick out the method definitions 
; by filtering on the first symbol (it should be method), and then call method-spec on each

(defn method-specs [sexprs]
    (->> sexprs
        (filter #(= 'method (first %)))
        (mapcat method-spec)
        (apply hash-map)))

; Example

(method-specs 
    '((method age [] (* 2 10)) 
      (method greet [visitor] (str "Hello there, " visitor))))


{:age (fn age [] (* 2 10)), :greet (fn greet [visitor] (str "Hello there, " visitor))}

In [98]:
; The modified version of 'new-class', which now accepts
; methods as args

(defn new-class [class-name methods]
    (fn klass [command & args]
        (case command
            :name (name class-name)
            :new (new-object klass))))


#'user/new-class

In [99]:
; The 'defclass' macro should be also modified 
; to accept method definitions
; (It doesn’t yet do anything with methods (yet))

(defmacro defclass [class-name & specs]
    (let [fns (or (method-specs specs) {})]
        `(def ~class-name (new-class '~class-name ~fns))))


#'user/defclass

In [100]:
; Now the class with methods can be defined as follows

(defclass Person
    (method age [] 
        (* 2 10))
    (method greet [visitor] 
        (str "Hello there, " visitor)))


#'user/Person

Invoking methods

In [106]:
; A modified 'new-class' function with a 'method' keyword
; that process an arbitrary method name

(defn find-method [method-name instance-methods]
    (instance-methods method-name))

(defn new-class [class-name methods]
    (fn klass [command & args]
        (case command
            :name (name class-name)
            :new (new-object klass)
            :method (let [[method-name] args]
                (find-method method-name methods)))))

; Example

(defclass Person
    (method age [] 
        (* 2 10))
    (method greet [visitor] 
        (str "Hello there, " visitor)))

(Person :method :age)


#function[user/age--4469]

In [107]:
; A modified 'new-object' with a default case 
; which will be assumed as a method call

(defn new-object [klass]
    (let [state (ref {})]
        (fn [command & args]
            (case command
                :class klass
                :class-name (klass :name)
                :set! (let [[k v] args] 
                    (dosync (alter state assoc k v))
                    nil)
                :get (let [[key] args]
                    (@state key))
                (if-let [method (klass :method command)]
                    (apply method args)
                    (throw (RuntimeException. (str "Unable to respond to " command))))))))


#'user/new-object

In [108]:
; Now an object can be created and methods can be invoked
; like traditional OOP languages

(def shelly (Person :new))


#'user/shelly

In [109]:
(shelly :age)


20

In [110]:
(shelly :greet "Nancy")


"Hello there, Nancy"

Referring to this

In [111]:
; Creating a dynamic var 'this' to avoid warnings
; about unresolved symbols

(declare ^:dynamic this)

#'user/this

In [112]:
; A modified 'new-object' with a default case 
; which now includes the binding of 'this' to 'thiz' (the function itself)

(defn new-object [klass]
    (let [state (ref {})]
    (fn thiz [command & args]
        (case command
            :class klass
            :class-name (klass :name)
            :set! (let [[k v] args] 
                (dosync (alter state assoc k v))
                nil)
            :get (let [[key] args] (@state key))
            (let [method (klass :method command)]
                (if-not method
                (throw (RuntimeException. (str "Unable to respond to " command))))
                (binding [this thiz]
                    (apply method args)))))))


#'user/new-object

In [113]:
; Now the class can be defined and the object instantiated, 
; as follows

(defclass Person
    (method age [] 
        (* 2 10))
    (method about [diff]
        (str "I was born about " (+ diff (this :age)) " years ago")))

(def shelly (Person :new))

(shelly :about 2)

"I was born about 22 years ago"

Class inheritance

In [123]:
; A function parses the specification of the parent class 
; from a given class definition

(defn parent-class-spec [sexprs]
    (let [extends-spec (filter #(= 'extends (first %)) sexprs)
          extends (first extends-spec)]
        (if (empty? extends)
            'OBJECT
            (last extends))))

(parent-class-spec '((extends Person) (method age [] (* 2 9))))


Person

In [124]:
; The 'defclass' macro now has to process the 'extends' keyword

(defmacro defclass [class-name & specs]
    (let [parent-class (parent-class-spec specs)
          fns (or (method-specs specs) {})]
        `(def ~class-name (new-class '~class-name #'~parent-class ~fns))))
        ; The reader macro for the var special form is #'


#'user/defclass

In [125]:
; The 'new-class' function has to accepy the parent class
; as a new arg

(defn new-class [class-name parent methods]
    (fn klass [command & args]
        (case command
            :name (name class-name)
            :parent parent
            :new (new-object klass)
            :method (let [[method-name] args] 
                (find-method method-name methods)))))


#'user/new-class

In [126]:
; Binding the name of OBJECT

(def OBJECT (new-class :OBJECT nil {}))


#'user/OBJECT

In [128]:
; Now the class and the subclass can be defined as follows

(defclass Person
    (method age [] 
        (* 2 10))
    (method about [diff]
        (str "I was born about " (+ diff (this :age)) " years ago")))

(defclass Woman (extends Person)
    (method greet [v] 
        (str "Hello, " v))
    (method age [] 
        (* 2 9)))


#'user/Woman

In [129]:
(def donna (Woman :new))


#'user/donna

In [130]:
(donna :greet "Shelly")


"Hello, Shelly"

In [131]:
(donna :age)


18

In [132]:
; This error shows since the 'about' method is defined in the parent class

(donna :about 3)


Execution error at user$new_object$thiz__4492/doInvoke (REPL:16).
Unable to respond to :about


class java.lang.RuntimeException: 

In [134]:
; To fix the last exception, the 'find-method' 
; looks up the method in the parent class, recursively

(defn find-method [method-name klass]
    (or ((klass :methods) method-name)
        (if-not (= #'OBJECT klass)
            (find-method method-name (klass :parent)))))

; Adjusting the 'new-class' function accordingly

(defn new-class [class-name parent methods]
    (fn klass [command & args]
        (case command
            :name (name class-name)
            :parent parent
            :new (new-object klass)
            :methods methods
            :method (let [[method-name] args] 
                (find-method method-name klass)))))


#'user/new-class

In [137]:
; Now the class and the subclass can be defined as follows

(defclass Person
    (method age [] 
        (* 2 10))
    (method about [diff]
        (str "I was born about " (+ diff (this :age)) " years ago")))

(defclass Woman (extends Person)
    (method greet [v] 
        (str "Hello, " v))
    (method age [] 
        (* 2 9)))

(def donna (Woman :new))


#'user/donna

In [139]:
; With the last changes, all the methods calls work as expected

(donna :about 3)


"I was born about 21 years ago"